In [1]:
import nidaqmx as ni

In [2]:
# initialize data arrays to hold results

In [93]:
import PyDAQmx.DAQmxFunctions as pydaqmxf
pydaqmxf.DAQmxResetDevice('Dev1')

0

In [4]:
import numpy as np

In [5]:
scan_frequency_ranges = [(2820e6, 2920e6, 101)] #change to start freq, stop freq and N step
#scan_frequency_ranges = [(30e6, 40e6, 1000)] #change to start freq, stop freq and N step

In [6]:
frequency_data = [np.linspace(*r) for r in scan_frequency_ranges]

In [7]:
#rename to samples per sweep? 

samples_per_line = sum(freq_range[-1] for freq_range in scan_frequency_ranges)

In [8]:
samples_per_line

101

In [9]:
run_time = 60
data_rate = 200  #200 samples/sec?
estimated_samples = run_time * data_rate

In [10]:
estimated_samples

12000

In [11]:
estimated_lines = max(1, int(1.05 * estimated_samples / samples_per_line)) #qudi adds a 5% buffer

In [12]:
estimated_lines #change to number of sweeps?

124

In [13]:
raw_data = dict()
fit_results = dict()
signal_data = dict()

In [14]:
raw_data['PFI0'] =  [
                np.full((freq_arr.size, estimated_lines), np.nan) for freq_arr in
                frequency_data
            ]


In [15]:
raw_data['PFI0'][0].shape

(101, 124)

In [16]:
# initialize the DAQ -- reproducing qudi

In [17]:
oversampling_factor = 1.0 #not sure why this is in qudi
sample_rate = oversampling_factor * data_rate

In [18]:
frequencies = scan_frequency_ranges[0]

In [19]:
samples = frequencies[-1]

In [91]:
mw_source.close()

In [20]:
import windfreak
mw_source = windfreak.SynthHD('COM5')
mw_direct_device = mw_source 


In [21]:
# mw_source.trigger_mode

In [22]:
# mw_source[0].sweep_enable = False

In [23]:
# mw_source[0].power = -5.0
# mw_source[0].frequency = 30e6

In [24]:
# mw_source[0].frequency

In [25]:
#turn MW source off
mw_direct_device._write('E0r0h0')

In [26]:
# mw_direct_device._read()

In [27]:
#turn MW source on
# mw_direct_device._write('E1r1h1')

In [28]:
# mw_source.close()

In [94]:
# trigger mode: single step
mw_direct_device._write('w0')

AttributeError: 'NoneType' object has no attribute 'write'

In [29]:
frequencies

(2820000000.0, 2920000000.0, 101)

In [30]:
# configures the MW source
scan_power = -5.0

# qudi_odmr_logic calls mw.configure_scan(power, frequencies, mode, sample_rate),
#which then calls
# mw._write_sweep, which is reproduced here
#start, stop, points = self._scan_frequencies
start, stop, points = frequencies

step = (stop - start) / (points - 1)

# sweep mode: linear sweep, non-continuous
# can later transform this into more readable windfreak calls
mw_direct_device._write('X0')
mw_direct_device._write('c0')

# trigger mode: single step
mw_direct_device._write('w2')

# sweep direction
if stop >= start:
    mw_direct_device._write('^1')
else:
    mw_direct_device._write('^0')

# sweep lower and upper frequency and steps
mw_direct_device._write(f'l{start / 1e6:5.7f}')
mw_direct_device._write(f'u{stop / 1e6:5.7f}')
mw_direct_device._write(f's{step / 1e6:5.7f}')

# set power
mw_direct_device._write(f'W{scan_power:2.3f}')
# set sweep lower end power
mw_direct_device._write(f'[{scan_power:2.3f}')
# set sweep upper end power
mw_direct_device._write(f']{scan_power:2.3f}')

#after _write_sweep, configure_sweep continues
mw_direct_device._write(f't{1000 * 0.75 / sample_rate:f}') 
scan_sample_rate = float(mw_direct_device._query('t?')) / 1000

print(f'Configured scan with: '
               f'scan_power = {scan_power}, '
               f'len(scan_frequencies) = {len(frequencies)}, '
               f'scan_sample_rate = {scan_sample_rate}')

Configured scan with: scan_power = -5.0, len(scan_frequencies) = 3, scan_sample_rate = 0.00375


In [31]:
frequencies

(2820000000.0, 2920000000.0, 101)

In [32]:
#start scan
mw_direct_device._write('g1')

In [33]:
def stat(a_mw_device):
    """ Return status of PLL, power amplifier and output power muting for current channel.

    @return tuple(bool): PLL on, power amplifier on, output power muting on
    """
    # PLL status
    E = int(a_mw_device._query('E?'))
    # power amplifier status
    r = int(a_mw_device._query('r?'))
    # hig/low power selector
    h = int(a_mw_device._query('h?'))
    return E, r, h

In [34]:
stat(mw_direct_device)

(0, 0, 0)

In [82]:
# NI DAQ setup

device_name = 'Dev1'
channels = ['PFI0']
rw_timeout = 10

In [35]:
device_handle = ni.system.Device(device_name)
all_counters = tuple(ctr.split('/')[-1] for ctr in device_handle.co_physical_chans.channel_names if 'ctr' in ctr.lower())

In [36]:
device_handle.reset_device()

In [37]:
system = ni.system.System.local()

In [38]:
#line 527 - daq.set_sample_rate(sample_rate) simply sets a property value of the DAQ software object.
#line 528 - daq.set_frame_size(samples) also sets a property value of the DAQ software object

# how do we start the NI DAQ card and how do we sync it to the frequency of the RF signal?

# qudi executes the _scan_odmr_line repeatedly until the total run time has elapsed
# recall that a 'line' here is like a raster-line in signal processing -- it's a single sweep through the full frequency range

#_scan_odrm_line
  # scanner = self._data_scanner()
  # new_counts = scanner.acquire_frame()
  # self._microwave().reset_scan() #why reset here when there are still tasks to perform? 

# scanner.acquire_frame()
    # self.start_buffered_acquisition()
    # data = self.get_buffered_samples(self._frame_size)
    # self.stop_buffered_acquisition()
    # return data

#scanner.start_buffered_acquisition()
#we do not have an external clock, so first a task is created to set up an internal clock


In [40]:
# WE MUST KEEP TRACK OF ALL TASK HANDLES
# THERE DOESN'T SEEM TO BE ANY WAY OF RETRIEVING THEM LATER
# UNCOMMENT BELOW ON FIRST RUN, COMMENT AGAIN TO PROTECT 

#all_tasks = {}


In [68]:
# the code sets up a sample clock task
task = None
for counter_name in all_counters:
    if task is None:
        task_name = 'SampleClock_{0:d}'.format(id(all_counters))
        task = ni.Task(task_name)

        task.co_channels.add_co_pulse_chan_freq(
            '/{0}/{1}'.format(device_name, counter_name),
            freq=sample_rate,
            idle_state=ni.constants.Level.LOW)
        task.timing.cfg_implicit_timing(
            sample_mode=ni.constants.AcquisitionType.FINITE,
            samps_per_chan=samples + 1)
        
        #task.control(ni.constants.TaskMode.TASK_RESERVE)
        
        print(f'created task {task_name} with counter {counter_name}')
        all_tasks[task_name] = task
                                

created task SampleClock_2454152451312 with counter ctr0


In [127]:
# for task_name, task in all_tasks.items():
#     task.stop()

{'SampleClock_3054461194304': Task(name=SampleClock_3054461194304)}

In [42]:
# we now set up tasks for each of the channels. Here we only use the digital channel pfi0

digital_channels = ['PFI0']

In [69]:
all_tasks

{'SampleClock_2454152451312': Task(name=SampleClock_2454152451312)}

In [70]:
clock_task_handle = all_tasks['SampleClock_2454152451312']
clock_task_handle.channel_names

['Dev1/ctr0']

In [71]:
digital_tasks = {} #keep track of digital_tasks
digital_readers =[] #keep track of digital read channels

In [72]:
import ctypes
import nidaqmx._lib  # Due to NIDAQmx C-API bug needed to bypass property getter
import nidaqmx.stream_readers

clock_channel = '/{0}InternalOutput'.format(clock_task_handle.channel_names[0])

for i, chnl in enumerate(digital_channels):
        chnl_name = '/{0}/{1}'.format(device_name, chnl)
        task_name = 'PeriodCounter_{0}'.format(chnl)
        task = ni.Task(task_name)
        # question -- DOES THE FOLLOWING ALWAYS NEED TO USE THE SAME COUNTER THAT WAS USED FOR THE CLOCK TASK ABOVE?
        # it seems that, from reading qudi, this will be the scenario. 
        # or maybe I need to use the next counter
        
        ctr_name = '/{0}/{1}'.format(device_name, 'ctr1') #MUST BE DIFFERENT COUNTER FROM ABOVE
        #ctr_name = f'/{clock_task_handle.channel_names[0]}'
        task.ci_channels.add_ci_period_chan(
                        ctr_name,
                        min_val=0,
                        max_val=100000000,
                        units=ni.constants.TimeUnits.TICKS,
                        edge=ni.constants.Edge.RISING)
        
        # following qudi here
        try:
            nidaqmx._lib.lib_importer.windll.DAQmxSetCIPeriodTerm(
                task._handle,
                ctypes.c_char_p(ctr_name.encode('ascii')),
                ctypes.c_char_p(clock_channel.encode('ascii')))
            nidaqmx._lib.lib_importer.windll.DAQmxSetCICtrTimebaseSrc(
                task._handle,
                ctypes.c_char_p(ctr_name.encode('ascii')),
                ctypes.c_char_p(chnl_name.encode('ascii')))
        except:
            print("exception")
            nidaqmx._lib.lib_importer.cdll.DAQmxSetCIPeriodTerm(
                task._handle,
                ctypes.c_char_p(ctr_name.encode('ascii')),
                ctypes.c_char_p(clock_channel.encode('ascii')))
            nidaqmx._lib.lib_importer.cdll.DAQmxSetCICtrTimebaseSrc(
                task._handle,
                ctypes.c_char_p(ctr_name.encode('ascii')),
                ctypes.c_char_p(chnl_name.encode('ascii')))

        task.timing.cfg_implicit_timing(
            sample_mode=ni.constants.AcquisitionType.FINITE,
            samps_per_chan=samples)
        
        #commenting out for now because it keeps throwing
        #task.control(ni.constants.TaskMode.TASK_RESERVE)
        
        digital_readers.append(nidaqmx.stream_readers.CounterReader(task.in_stream))
        digital_readers[-1].verify_array_shape = False
        digital_tasks[task_name] = task

In [47]:
#task.close()
#pydaqmxf.DAQmxClearTask(task._handle)

In [73]:
digital_tasks

{'PeriodCounter_PFI0': Task(name=PeriodCounter_PFI0)}

In [80]:
digital_readers

In [74]:
for task_name, task in digital_tasks.items():
    task.start()

In [65]:
# for task_name, task in digital_tasks.items():
#     task.close()

In [51]:
pydaqmxf.DAQmxResetDevice('Dev1')

0

In [86]:
#digital_tasks['PeriodCounter_PFI0'].start()

In [75]:
clock_task_handle.start() #reset the device if this throws

In [77]:
# qudi - get_buffered_samples
digital_tasks['PeriodCounter_PFI0'].in_stream.avail_samp_per_chan  #these are the number of samples available to be read out

101

In [81]:
samples

101

In [78]:
data_buffer = np.zeros(samples)

In [79]:
data_buffer

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [83]:
reader = digital_readers[-1]

read_samples = reader.read_many_sample_double(
                    data_buffer,
                    number_of_samples_per_channel=samples,
                    timeout=rw_timeout)

In [84]:
data = dict()

In [85]:
data_buffer

array([3., 0., 2., 1., 1., 1., 1., 1., 3., 2., 1., 2., 1., 2., 3., 3., 2.,
       2., 2., 1., 1., 0., 1., 0., 4., 1., 0., 2., 2., 2., 0., 2., 3., 0.,
       3., 1., 4., 1., 4., 3., 2., 3., 4., 1., 2., 2., 1., 1., 1., 5., 5.,
       2., 1., 1., 4., 5., 0., 2., 2., 1., 1., 1., 2., 0., 1., 1., 1., 2.,
       2., 1., 1., 2., 2., 1., 1., 0., 2., 3., 2., 0., 1., 1., 0., 2., 1.,
       0., 5., 0., 0., 1., 0., 3., 1., 1., 0., 0., 1., 1., 2., 1., 0.])

In [86]:
data[reader._task.name.split('_')[-1]] = data_buffer

In [87]:
data

{'PFI0': array([3., 0., 2., 1., 1., 1., 1., 1., 3., 2., 1., 2., 1., 2., 3., 3., 2.,
        2., 2., 1., 1., 0., 1., 0., 4., 1., 0., 2., 2., 2., 0., 2., 3., 0.,
        3., 1., 4., 1., 4., 3., 2., 3., 4., 1., 2., 2., 1., 1., 1., 5., 5.,
        2., 1., 1., 4., 5., 0., 2., 2., 1., 1., 1., 2., 0., 1., 1., 1., 2.,
        2., 1., 1., 2., 2., 1., 1., 0., 2., 3., 2., 0., 1., 1., 0., 2., 1.,
        0., 5., 0., 0., 1., 0., 3., 1., 1., 0., 0., 1., 1., 2., 1., 0.])}

In [89]:
#then qudi stops all tasks


clock_task_handle.stop() #reset the device if this throws
clock_task_handle.close() #reset the device if this throws
clock_task_handle = None

In [90]:
for task_name, task in digital_tasks.items():
    task.stop()
    task.close()

In [92]:
read_samples

101